In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter

# --- 1. ЗАВАНТАЖЕННЯ ТА ОБРОБКА ДАНИХ ---

def process_data(file_paths):
    """
    Зчитує список файлів (технічна документація) та повертає список слів.
    """
    all_words = []

    # Якщо передано один рядок замість списку, робимо його списком
    if isinstance(file_paths, str):
        file_paths = [file_paths]

    for file_name in file_paths:
        try:
            with open(file_name, "r", encoding="utf-8") as f:
                text = f.read().lower()
                # Використовуємо регулярний вираз для вилучення слів.
                # Для технічної документації можна додати ігнорування спецсимволів коду,
                # але \w+ є універсальним базовим варіантом.
                words = re.findall(r'\w+', text)
                all_words.extend(words)
        except FileNotFoundError:
            print(f"Помилка: Файл {file_name} не знайдено.")

    return all_words

def get_count(word_l):
    """ Створює словник частотності слів. """
    return Counter(word_l)

def get_probs(word_count_dict):
    """ Обчислює ймовірності слів P(w). """
    probs = {}
    total_count = sum(word_count_dict.values())
    for word, count in word_count_dict.items():
        probs[word] = count / total_count
    return probs

# --- 2. ОПЕРАЦІЇ РЕДАГУВАННЯ ---

def delete_letter(word):
    """ Видаляє одну літеру. Example: 'nice' -> 'ice', 'nce', 'nie', 'nic' """
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    delete_l = [L + R[1:] for L, R in split_l if R]
    return delete_l

def switch_letter(word):
    """ Міняє місцями дві сусідні літери. Example: 'eta' -> 'tea' """
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    switch_l = [L + R[1] + R[0] + R[2:] for L, R in split_l if len(R) >= 2]
    return switch_l

def replace_letter(word):
    """ Замінює одну літеру на іншу. """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    replace_l = [L + c + R[1:] for L, R in split_l if R for c in letters if c != R[0]]
    return replace_l

def insert_letter(word):
    """ Вставляє одну літеру. """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    insert_l = [L + c + R for L, R in split_l for c in letters]
    return insert_l

# --- 3. ПОШУК КАНДИДАТІВ (РЕДАГУВАННЯ 1 та 2 РІВНІВ) ---

def edit_one_letter(word, allow_switches=True):
    """ Усі слова на відстані 1 редагування. """
    edit_set = set()
    edit_set.update(delete_letter(word))
    if allow_switches:
        edit_set.update(switch_letter(word))
    edit_set.update(replace_letter(word))
    edit_set.update(insert_letter(word))
    return edit_set

def edit_two_letters(word, allow_switches=True):
    """ Усі слова на відстані 2 редагувань. """
    edit_set = set()
    edit_one = edit_one_letter(word, allow_switches)
    for w in edit_one:
        if w:
            edit_set.update(edit_one_letter(w, allow_switches))
    return edit_set

# --- 4. АВТОКОРЕКЦІЯ ---

def get_corrections(word, probs, vocab, n=2, verbose=False):
    """
    Основна функція автокорекції.
    Input: word (слово з помилкою), probs (ймовірності), vocab (словник), n (кількість топ-пропозицій)
    Output: список кортежів (слово, ймовірність)
    """
    suggestions = []

    # Крок 1: Перевіряємо, чи слово вже є у словнику
    if word in vocab:
        suggestions = [word]
    else:
        # Крок 2: Перевіряємо слова на відстані 1 редагування
        edit_one_set = edit_one_letter(word)
        suggestions_one = edit_one_set.intersection(vocab)

        if suggestions_one:
            suggestions = list(suggestions_one)
        else:
            # Крок 3: Перевіряємо слова на відстані 2 редагувань
            edit_two_set = edit_two_letters(word)
            suggestions_two = edit_two_set.intersection(vocab)

            if suggestions_two:
                suggestions = list(suggestions_two)
            else:
                # Якщо нічого не знайдено, повертаємо саме слово
                suggestions = [word]

    # Сортуємо за ймовірністю
    best_words = {}
    for w in suggestions:
        best_words[w] = probs.get(w, 0)

    n_best = sorted(best_words.items(), key=lambda x: x[1], reverse=True)[:n]

    if verbose:
        print(f"Введене слово: {word}")
        print(f"Кандидати: {n_best}")

    return n_best

# --- 5. МІНІМАЛЬНА ВІДСТАНЬ РЕДАГУВАННЯ (Dynamic Programming) ---

def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    """
    Матричний розрахунок мінімальної відстані редагування (алгоритм Вагнера-Фішера).
    """
    m = len(source)
    n = len(target)

    # Створення матриці (m+1) x (n+1)
    D = np.zeros((m+1, n+1), dtype=int)

    # Ініціалізація нульового рядка та стовпця
    for row in range(1, m + 1):
        D[row, 0] = D[row-1, 0] + del_cost

    for col in range(1, n + 1):
        D[0, col] = D[0, col-1] + ins_cost

    # Заповнення матриці
    for row in range(1, m + 1):
        for col in range(1, n + 1):

            r_cost = rep_cost

            if source[row - 1] == target[col - 1]:
                r_cost = 0

            D[row, col] = min(
                D[row - 1, col] + del_cost,      # Видалення
                D[row, col - 1] + ins_cost,      # Вставка
                D[row - 1, col - 1] + r_cost     # Заміна
            )

    med = D[m, n]
    return D, med

# --- 6. ЗАПУСК ТА ДЕМОНСТРАЦІЯ (ДЛЯ ВАРІАНТУ 7) ---

# Для демонстрації варіанту 7 я створю імітацію корпусу технічної документації.
# В реальній роботі ви завантажите сюди .rst або .md файли з readthedocs-examples.
technical_corpus_simulation = """
Python is an interpreted high-level general-purpose programming language.
Its design philosophy emphasizes code readability with its use of significant indentation.
Its language constructs as well as its object-oriented approach aim to help programmers
write clear, logical code for small and large-scale projects.
Documentation is crucial for software development.
Functions, variables, classes, and modules are core components.
Installation instructions: pip install requirements.
Configuration files are usually located in the root directory.
Deployment to production servers requires careful testing.
Exception handling is done via try-except blocks.
"""

# Зберігаємо імітацію у файл для роботи функцій
with open("technical_docs.txt", "w", encoding="utf-8") as f:
    f.write(technical_corpus_simulation)

# 1. Створення словника
word_list = process_data("technical_docs.txt")
# Можна також додати ваш файл shakespeare.txt для розширення словника:
# word_list += process_data("shakespeare.txt")

vocab = set(word_list)
word_counts = get_count(word_list)
probs = get_probs(word_counts)

print(f"Кількість унікальних слів у словнику: {len(vocab)}")
print("-" * 30)

# 2. Тестування Автокорекції на технічних термінах
tech_misspellings = [
    'pyton',       # python
    'functon',     # function (в нашому тексті functions)
    'variabl',     # variable (в нашому тексті variables)
    'instalation', # installation
    'cod',         # code
    'projcts'      # projects
]

print("--- Результати автокорекції ---")
for w in tech_misspellings:
    corrections = get_corrections(w, probs, vocab, n=3)
    print(f"Original: {w:15} -> Correction: {corrections}")

print("\n" + "-" * 30 + "\n")

# 3. Тестування Min Edit Distance
print("--- Розрахунок мінімальної відстані редагування ---")
source = "intention"
target = "execution"
matrix, dist = min_edit_distance(source, target)

print(f"Відстань між '{source}' та '{target}': {dist}")
print("Матриця відстаней:")
df = pd.DataFrame(matrix, index=['#'] + list(source), columns=['#'] + list(target))
print(df)

# Приклад для технічних термінів
source_tech = "moudle"
target_tech = "module"
_, dist_tech = min_edit_distance(source_tech, target_tech)
print(f"\nВідстань між '{source_tech}' та '{target_tech}': {dist_tech}")

Кількість унікальних слів у словнику: 77
------------------------------
--- Результати автокорекції ---
Original: pyton           -> Correction: [('python', 0.011235955056179775)]
Original: functon         -> Correction: [('functions', 0.011235955056179775)]
Original: variabl         -> Correction: [('variables', 0.011235955056179775)]
Original: instalation     -> Correction: [('installation', 0.011235955056179775)]
Original: cod             -> Correction: [('code', 0.02247191011235955)]
Original: projcts         -> Correction: [('projects', 0.011235955056179775)]

------------------------------

--- Розрахунок мінімальної відстані редагування ---
Відстань між 'intention' та 'execution': 8
Матриця відстаней:
   #  e  x   e   c   u   t   i   o   n
#  0  1  2   3   4   5   6   7   8   9
i  1  2  3   4   5   6   7   6   7   8
n  2  3  4   5   6   7   8   7   8   7
t  3  4  5   6   7   8   7   8   9   8
e  4  3  4   5   6   7   8   9  10   9
n  5  4  5   6   7   8   9  10  11  10
t  6  5  